# Travel Demand Model Data Inputs
> 

## Setup 

In [ ]:
import pandas as pd
import pathlib
import os
from utils import *

In [ ]:
# current working directory and git repository path
local_path = pathlib.Path().absolute()
repo_path = pathlib.Path(local_path).parents[1]
# set workspace
arcpy.env.workspace = os.path.join(local_path, 'Workspace.gdb')
# overwrite true
arcpy.env.overwriteOutput = True

In [ ]:
print('Current working directory:', local_path)
print('Repository directory:', repo_path)

#### Notes
* old inputs: F:\Research and Analysis\misc\Reid_Haefer\Model\model_update_2018\data_inputs\lodging_occupancy
* Final inputs to produce: F:\Transportation\model\scenario_base\zonal

### Base Data

In [ ]:
# Get TAZ data
taz_url = 'https://maps.trpa.org/server/rest/services/Transportation_Planning/MapServer/6'
sdf_taz = get_fs_data_spatial(taz_url)

# place holder URL - Demographics URL will have a new Feature Service titled "Block Group 2022 - Travel Demand Model Inputs"
census_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/1'
sdf_census = get_fs_data_spatial(census_url)

# Get VHR data - this is current, no way to get historical data outside of the City of South Lake Tahoe
vhr_url = 'https://maps.trpa.org/server/rest/services/VHR/MapServer/0'
sdf_vhr = get_fs_data_spatial(vhr_url)

# Get Cumulative Accounting Unit data
units_url = 'https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2'
sdf_units = get_fs_data_spatial_query(units_url, "Year = 2022")


In [ ]:
# Get Cumulative Accounting Unit data
units_url = 'https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2'
sdf_units = get_fs_data_spatial_query(units_url, "Year = 2022")

sdf_units.to_csv('cumulative_accounting_units_2022.csv', index=False)

In [ ]:
sdf_units.spatial.to_featureclass(os.path.join(local_path, 'Workspace.gdb','cumulative_accounting_units_2022'))

In [ ]:
print(local_path)

## Socio Econ
> TAZ by total_residential_units,census_occ_rate,total_occ_units,occ_units_low_inc,occ_units_med_inc,occ_units_high_inc,persons_per_occ_unit,total_persons,emp_retail,emp_srvc,emp_rec,emp_game,emp_other

In [ ]:
units_url = ''
sdf_res = get_fs_data_spatial_query(units_url, "YEAR = 2022")

### Employment

> Purpose: Assign employees to TAZs
* Data Axle data and CBP data sources

In [ ]:
employment_url = ''

### Income by Residential Unit

> Purpose: Calculate #/% of residential units in low/mid/high income in each TAZ
* Get # of HH in each income bin by census block group from ACS
* Combine to low/mid/high income, with breaks at 60k and 100k
* Assign to TAZ


In [ ]:
# place holder URL - Demographics URL will have a new Feature Service titled "Block Group 2022 - Travel Demand Model Inputs"
census_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/1'
sdf_census = get_fs_data_spatial(census_url)


## Overnight Visitation
>TAZ by hotelmotel,resort,casino,campground,percentHouseSeasonal,beach

In [ ]:
# output to csv that matches:
df_visit = pd.read_csv(os.path.join(repo_path,r"TravelDemandModel\2022\data\model_inputs\OvernightVisitorZonalData_Summer.csv"))
df_visit.info()

### Tourist Accommodation Units

In [ ]:
# # Get VHR data - this is current, no way to get historical data outside of the City of South Lake Tahoe
# vhr_url = 'https://maps.trpa.org/server/rest/services/VHR/MapServer/0'
# sdf_vhr = get_fs_data_spatial(vhr_url)

# Get Tourist Accomodation Units data
tau_url = 'https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2'
sdf_tau = get_fs_data_spatial_query(tau_url, "Year = 2022")

# Get TAZ data
taz_url = 'https://maps.trpa.org/server/rest/services/Transportation_Planning/MapServer/6'
sdf_taz = get_fs_data_spatial(taz_url)

In [ ]:
# set TAU_Type to null text field
sdf_tau['TAU_Type'] = None
# columns to keep
tau_columns = ['APN',
                'TouristAccommodation_Units', 'YEAR',
                'JURISDICTION', 'COUNTY', 'OWNERSHIP_TYPE',
                'EXISTING_LANDUSE', 'TAZ', 'TAU_Type',
                'WITHIN_TRPA_BNDY', 'PARCEL_ACRES', 'PARCEL_SQFT', 'SHAPE']
# filter to columns
sdf_tau = sdf_tau[tau_columns]

# get only parcels with tourist accomodation units
sdf_tau = sdf_tau[sdf_tau['TouristAccommodation_Units'] > 0]
sdf_tau.info()

In [ ]:
# casino or resort parcels
casino_apns = [# south shore
                '1318-27-002-002','1318-27-002-005','1318-27-001-009','1318-27-001-021'
               # north shore
               '123-031-01', '123-042-01', '123-052-04', '123-042-15']

resort_apns = [# LT resort, hyatt, hilton, >?
                '127-280-02','029-480-004','027-090-025']

# set TAU_Type feild values to Hotel/Motel, Casino, or Resort
sdf_tau['TAU_Type'] = "Hotel/Motel"
# if the APN is in the list of casino APNs, set the TAU Type to Casino
sdf_tau.loc[sdf_tau['APN'].isin(casino_apns), 'TAU_Type'] = "Casino"
# if the APN is in the list of resort APNs, set the TAU Type to Resort
sdf_tau.loc[sdf_tau['APN'].isin(resort_apns), 'TAU_Type'] = "Resort"

sdf_tau.info()


In [ ]:

# spatial join TAZ data to TAU data
arcpy.SpatialJoin_analysis(sdf_tau, sdf_taz, 'taz_tau', 
                           'JOIN_ONE_TO_ONE', 'KEEP_ALL', 
                           match_option='HAVE_THEIR_CENTER_IN')
# read in output of spatial join as sdf
sdf_tau_taz = pd.DataFrame.spatial.from_featureclass('taz_tau')
# set TAZ = to TAZ_1
sdf_tau_taz['TAZ'] = sdf_tau_taz['TAZ_1']
# keep columns
sdf_tau_taz = sdf_tau_taz[tau_columns]

# group by TAU_Type and sum of TouristAccommodation_Units within TAZ
sdf_tau_taz_grouped = sdf_tau_taz.groupby(['TAU_Type', 'TAZ']).agg(
                                                {'TouristAccommodation_Units': 'sum'}).reset_index()
# unstack by TAU_Type as columns and TAZ as a column
sdf_tau_taz_grouped_pivot = sdf_tau_taz_grouped.pivot(index='TAZ', 
                                                      columns='TAU_Type', 
                                                      values='TouristAccommodation_Units').reset_index()
# merge to sdf_taz to get all tazs
sdf_taz_tau = pd.merge(sdf_taz, sdf_tau_taz_grouped_pivot, how='left', on='TAZ')

# rename columns
sdf_taz_tau.rename(columns={'Hotel/Motel':'hotelmotel',
                            'Casino':'casino',
                            'Resort':'resort'}, inplace=True)

# set columns
sdf_taz_tau['campground'] = 0
sdf_taz_tau['percentHouseSeasonal'] = 0

# keep only columns of interest
sdf_taz_tau = sdf_taz_tau[['TAZ', 'hotelmotel', 'casino', 'resort','campground','percentHouseSeasonal']]
sdf_taz_tau = sdf_taz_tau.fillna(0)
# cast all fields to int
sdf_taz_tau = sdf_taz_tau.astype(int)

# export to csv
sdf_taz_tau.to_csv(os.path.join('OvernightVisitorZonalData_Summer.csv'), index=False)


In [ ]:
# get total TAUs by TAU_Type
sdf_tau_type = sdf_tau.groupby('TAU_Type').agg({'TouristAccommodation_Units':'sum'}).reset_index()
# add total row
sdf_tau_type.loc['Total'] = sdf_tau_type.sum(numeric_only=True)

sdf_tau_type

### Campgrounds

> Purpose: calculate campground occupancy on model day and assign to TAZs
* in-process data: "F:\Research and Analysis\Transportation\Travel_Demand_Model\2023 Update\Input Data\Campgrounds\Campground_Visitation.xlsx"
* USFS data: https://apps.fs.usda.gov/arcx/rest/services/EDW/EDW_RecreationAreaActivities_01/MapServer/0

In [ ]:
# Get the data
campgroundu_url = 'https://maps.trpa.org/server/rest/services/Recreation/MapServer/1'
# campgrounds
sdf_campgrounds = get_fs_data_spatial_query(campgroundu_url, "RECREATION_TYPE = 'Campground")

# campground visitation
df_campground_visitation = pd.read_csv('2022/Data/Campground_Visitation.csv')

In [ ]:
# CA SP info https://csparks.maps.arcgis.com/apps/webappviewer/index.html?id=f96a883ff4154455b23bdc119f4574a9
# CA SP visitation

In [ ]:
import pandas as pd
import requests
# get data params with Mason's API key
campsite_url = 'https://ridb.recreation.gov/api/v1/facilities/232768/campsites'
api = '45310cc1-df4f-4824-88ba-8bd75d06f57e'
params = {'apikey': api}

# rec data from https://ridb.recreation.gov/download
# https://ridb.recreation.gov/downloads/reservations2022.zip
# local copy
rec_gov = r"C:\Users\mbindl\Downloads\reservations2022\FY22 Historical Reservations Full.csv"
df_res = pd.read_csv(rec_gov)

In [ ]:
# empty data frame to store campsite info
df_campsite_info_all = pd.DataFrame(columns=['Campground','Campsites', 'Total Nights', 'Total Visitors'])

# get campsite info from recreation.gov api
campgound_ids = {'232768':'Nevada Beach Campground',
                 '232769':'Fallen Leaf Campground',
                 '232876':'Meeks Bay Campground',
                 '232874':'William Kent Campground',
                 '232875':'Kaspian Campground',}

# create loop of campgrounds ids and get requests and count campsites
for campsite_id, campsite_name in campgound_ids.items():
    campsite_url = f'https://ridb.recreation.gov/api/v1/facilities/{campsite_id}/campsites'
    # get data for "Nevada Beach Campground"
    campsite_data = requests.get(campsite_url, params=params).json()
    # parse the data into a dataframe
    df_campsite = pd.DataFrame(campsite_data['RECDATA'])
    # count campsites
    print(f'{campsite_name} has {df_campsite.CampsiteName.count()} campsites')
    site_count = df_campsite.CampsiteName.count()

    # filter by campground id from the downloaded data
    df = df_res[df_res.facilityid == int(campsite_id)]
    # filter to usetype = Overnight
    df = df[df.usetype == 'Overnight']
    # filter by August 2022
    df_august22 = df[df.startdate.str.contains('2022-08')]
    # # filter out cancelled reservations
    # df_august22 = df_august22[df_august22.status != 'Cancelled']
    # # filter out reservations with 0 nights
    # df_august22 = df_august22[df_august22.nights != '0']
    # filter out reservations with end date in September 2022
    df_august22 = df_august22[~df_august22.enddate.str.contains('2022-09')]
    # use .loc to strip off ' days' from the nights column
    df_august22.loc[:, 'nights'] = df_august22.nights.map(lambda x: x.rstrip(' days'))
    # use .loc to strip off ' day' from the nights column
    df_august22.loc[:, 'nights'] = df_august22.nights.map(lambda x: x.rstrip(' day'))
    # drop a specific row based on a value in nights column
    df_august22 = df_august22[df_august22.nights != '00:00:00']
    # convert nights column to integer
    df_august22.nights = df_august22.nights.astype(int)

    # number of nights reserved for Nevada Beach Campground in August 2022
    total_nights = df_august22.nights.sum()
    # get total number of people reserved for Nevada Beach Campground in August 2022
    total_people = df_august22.numberofpeople.sum()
    print(f'{campsite_name} had {total_nights} nights reserved and {total_people} total visitors during August 2022')
    # append as row to new dataframe with campsite name, total nights, and total people
    df_campsite_info = pd.DataFrame([[campsite_name, site_count, total_nights, total_people]], 
                                    columns=['Campground', 'Campsites','Total Nights', 'Total Visitors'])
    
    # Check if 'df_campsite_info_all' exists in the local namespace
    if 'df_campsite_info_all' in locals():
        # If it exists, append 'df_campsite_info' to it as a new row
        df_campsite_info_all = pd.concat([df_campsite_info_all, df_campsite_info], ignore_index=True)
    else:
        # If it doesn't exist, assign 'df_campsite_info' to it directly
        df_campsite_info_all = df_campsite_info.copy()

In [ ]:
# create loop of campgrounds ids and get requests and count campsites
for campsite_id, campsite_name in campgound_ids.items():
    campsite_url = f'https://ridb.recreation.gov/api/v1/facilities/{campsite_id}/campsites'
    # get data for "Nevada Beach Campground"
    campsite_data = requests.get(campsite_url, params=params).json()
    # parse the data into a dataframe
    df_campsite = pd.DataFrame(campsite_data['RECDATA'])
    # count campsites
    print(f'{campsite_name} has {df_campsite.CampsiteName.count()} campsites')
    site_count = df_campsite.CampsiteName.count()

    # filter by campground id from the downloaded data
    df = df_res[df_res.facilityid == int(campsite_id)]
    # filter to usetype = Overnight
    df = df[df.usetype == 'Overnight']
    # filter by August 2022
    df_august22 = df[df.startdate.str.contains('2022-08')]

In [ ]:
df_august22.info()

In [ ]:
df_campsite_info_all


In [ ]:
df

In [ ]:
# remove any reservation that has an enddate in september
df_res_august = df_res[~df_res.enddate.str.contains('2022-09')]

## Occupancy Rates
> TAZ by hotelmotel,resort,casino,campground,house,seasonal

### Lodging Occupancy


> Purpose: calculate occupancy rates for hotels and VHRs and assign to TAZs

* Placer has Occupancy data at the TOT tax district. Josh will get the spatial file from Placer County GIS. 
* City of South Lake has hotel level occupancy data that needs to be converted to a spatial file. Reports live here: https://www.cityofslt.us/805/Zone-Detail-Reports 
* Douglas County has occupancy rates from the Casino reports: https://gaming.nv.gov/about/abstract/report/
* Washoe County has occupancy rates at the District level from RSCVA (single number by type of occupancy) "F:\Research and Analysis\Visitation\occupancy\Washoe B Occupied Rooms by Market Segment - 2022.xlsx"
* Rest of El Dorado County? is only VHRs, estimating Occupancy Rates based on TOT rates F:\Research and Analysis\Visitation\occupancy

In [ ]:
# Get VHR data - this is current, no way to get historical data outside of the City of South Lake Tahoe
vhr_url = 'https://maps.trpa.org/server/rest/services/VHR/MapServer/0'
sdf_vhr = get_fs_data_spatial(vhr_url)

# Get Tourist Accomodation Units data
tau_url = 'https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2'
sdf_tau = get_fs_data_spatial_query(tau_url, "Year = 2022")

# get occupancy rate data - see above for sources
df_occupancy = pd.read_csv('TravelDemandModel\2022\data\Occupancy_Rate.csv')

# get occupance rate zone data
sdf_occupancy_zones = get_fs_data_spatial('https://maps.trpa.org/server/rest/services/Travel_Demand_Model/MapServer/1')


### Housing Occupancy


> Purpose: estimate household size in each occupied housing unit
* get total Residential Units aggregated to TAZ
* get total VHRs aggregated to TAZ
* apply occupancy rate from ACS
* ACS 2022 by Block Group - household size

In [ ]:
# Get Houshold Block Group data
household_url = ''
sdf_household = get_fs_data_spatial(household_url)

## School Enrollment
> TAZ by elementary_school_enrollment,middle_school_enrollment,high_school_enrollment,college_enrollment

> Purpose: Collate school enrollment files
* create spatial file 

In [ ]:
# Get School Enrollment data
school_url_table     = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/32'
df_school_enrollment = get_fs_data(school_url_table)

# Get School Enrollment data - spatial
school_url_spatial = 'https://maps.trpa.org/server/rest/services/Datadownloader_PlanningandJurisdictions/MapServer/14'
sdf_school         = get_fs_data_spatial(school_url_spatial)

# Get TAZ data
taz_url = 'https://maps.trpa.org/server/rest/services/Transportation_Planning/MapServer/6'
sdf_taz = get_fs_data_spatial(taz_url)

In [ ]:
# set Type to Null
sdf_school['TYPE'] = None
# set SchoolType to 'elementary' if it contains 'elementary' or 'magnet' or 'academy'
sdf_school.loc[sdf_school['NAME'].str.contains('elementary', case=False), 'TYPE'] = 'Elementary School'
# set SchoolType to 'middle' if it contains 'middle'
sdf_school.loc[sdf_school['NAME'].str.contains('middle', case=False), 'TYPE'] = 'Middle School'
# set SchoolType to 'high' if it contains 'high'
sdf_school.loc[sdf_school['NAME'].str.contains('high', case=False), 'TYPE'] = 'High School'
# set SchoolType to 'college' if it contains 'college'
sdf_school.loc[sdf_school['NAME'].str.contains('college', case=False), 'TYPE'] = 'College'
# set SchoolType to 'other' if it it does not contain any of the above
sdf_school.loc[sdf_school['TYPE'].isnull(), 'TYPE'] = 'Elementary School'

In [ ]:
# spatial join TAZs to School points
sdf_school_taz = sdf_school.spatial.join(sdf_taz, how='inner')
# group by TYPE and sum of Enrollment within TAZ 
sdf_school_taz_grouped = sdf_school_taz.groupby(['TYPE', 'TAZ']).agg(
                                                {'ENROLLMENT': 'sum'}).reset_index()
# unstack by TYPE as columns and TAZ as a column
sdf_school_taz_grouped_pivot = sdf_school_taz_grouped.pivot(index='TAZ', 
                                                            columns='TYPE', 
                                                            values='ENROLLMENT').reset_index()
# merge to sdf_taz to get all tazs
sdf_taz_school = pd.merge(sdf_taz, sdf_school_taz_grouped_pivot, how='left', on='TAZ')
# drop SHAPE column
sdf_taz_school = sdf_taz_school.drop(columns='SHAPE')
# fill NA with 0 for all rows
sdf_taz_school = sdf_taz_school.fillna(0)
# cast all fields to int
sdf_taz_school = sdf_taz_school.astype(int)
# rename columns
sdf_taz_school.rename(columns={'Elementary School':'elementary_school_enrollment',
                               'Middle School':'middle_school_enrollment',
                               'High School':'high_school_enrollment',
                               'College':'college_enrollment'}, inplace=True)

# export to csv
sdf_taz_school.to_csv(os.path.join('SchoolEnrollment.csv'), index=False)

In [ ]:
sdf_taz_school

#### other way using arcpy spatial join and field mappings 'sum' feature

In [ ]:
df_school_enrollment_22 = df_school_enrollment[df_school_enrollment['Year'] == '2022-2023']
# Add a row for LTCC - Lake Tahoe Community College
ltcc = {'School_Name': 'Lake Tahoe Community College', 'Level_': 'College', 'Enrollment': 2909}
df_school_enrollment_22 = pd.concat([df_school_enrollment_22, pd.DataFrame([ltcc])], ignore_index=True)
# join school spatial to school table
sdf_school_enroll = pd.merge(sdf_school, df_school_enrollment, left_on='SchoolID', how='left')

In [ ]:
# Get TAZ data
taz_url = 'https://maps.trpa.org/server/rest/services/Transportation_Planning/MapServer/6'
sdf_taz = get_fs_data_spatial(taz_url)

# Get School Enrollment data - spatial
school_url_spatial = 'https://maps.trpa.org/server/rest/services/Datadownloader_PlanningandJurisdictions/MapServer/14'
sdf_school         = get_fs_data_spatial(school_url_spatial)


# keep only the columns we need; SHAPE and TAZ for TAZ and SHAPE and ENROLLMENT for schools
sdf_taz = sdf_taz[['TAZ', 'SHAPE']]
sdf_school = sdf_school[['ENROLLMENT', 'SHAPE']]

# convert to feature class
sdf_taz.spatial.to_featureclass(location=os.path.join(arcpy.env.workspace, 'TAZ'), overwrite=True, sanitize_columns=False)
sdf_school.spatial.to_featureclass(location=os.path.join(arcpy.env.workspace, 'Schools'), overwrite=True, sanitize_columns=False)

In [ ]:
field_mappings = arcpy.FieldMappings()
# add tables to field mappings
field_mappings.addTable('TAZ')
field_mappings.addTable('Schools')


# print names of fields in field mappings
for field in field_mappings.fields:
    print(field.name)

In [ ]:
# create field mappings
field_mappings = arcpy.FieldMappings()

# add tables to field mappings
field_mappings.addTable('TAZ')
field_mappings.addTable('Schools')

# for the output.
enrollment = field_mappings.findFieldMapIndex("ENROLLMENT")
fieldmap = field_mappings.getFieldMap(enrollment)
 
# Get the output field's properties as a field object
field = fieldmap.outputField
 
# Rename the field and pass the updated field object back into the field map
field.name = "Total_Enrollment"
field.aliasName = "Total Enrollment"
fieldmap.outputField = field
 
# Set the merge rule to mean and then replace the old fieldmap in the mappings object
# with the updated one
fieldmap.mergeRule = "sum"
field_mappings.replaceFieldMap(enrollment, fieldmap)

# spatial join that sums enrollment for each TAZ
arcpy.analysis.SpatialJoin(
    target_features   =sdf_taz,
    join_features     =sdf_school,
    out_feature_class ="TAZ_School_Enrollment",
    join_operation    ="JOIN_ONE_TO_ONE",
    join_type         ="KEEP_ALL",
    field_mapping     =field_mappings,
    match_option      ="INTERSECT",
)

In [ ]:
# get the data 
sdf_taz_school = pd.DataFrame.spatial.from_featureclass("TAZ_School_Enrollment")



## Forecasts
> Forecast 2040 and 2050 development and population change

In [ ]:
# total_residential_units - base_2018 
# forecast will be for 2040 and 2050 
# rate of development will be based on the current rate of development from the last 12 years (back to 2012)
    # current rate will not get us to full build out and will be adjusted to get to full build out by 2050
# total_residential_units = base_2018 + (rate_of_development * (2040 - 2018))

# forecast max build out will be 2050
#  still going to build out all the residential units and then revisit how conversions of TAUs and CFA will be handled

# GIS exercise of where the new residential units will be built
# 1. get the land use data and see if we can get the residential units on vacant and underbuilt parcels

# For TAUs and CFA we only built out what was in the pipeline

# Total Occupied Units = Total Residential Units - Vacant Units
    # based on block group rate and TAZ crosswalk assigned to Parcel level units
# Occupied Units by Income Level = Total Occupied Units * % of Income Level in Block Group
    # based on block group rate and TAZ crosswalk assigned to Parcel level units

# Lodging Occupany Rates by Tax Rate Zone
    # Air DNA? for VHR occupancy rates
    # Seasonal Units will be based on the % of seasonal units in the block group?

# Adjusted occupancy rates for Residential units to be based on population change to decennial census
    # double check total persons in the model against the decennial census population and then apply the rate?

# use adjusted ACS numbers to make all the input factors match the same source
    # use the 2022 ACS data at the Basin level for all the input factors
        # Block Group level data will be to noisy and not as accurate as the Basin level data

# forecast growth at the Basin level and show some population growth...
    # out year will be 2050 and show the growth in the model at 0.5% per year
    # show the growth in the model at 1.0% per year? or use the decennial census growth rate?
        # which was 0.04% per year from 2010 to 2020 annualized

# show the growth in the model at 0.004% per year? or use the decennial census growth rate?
    # adding 3000 units of affordable housing in the model and get 6,000 person increase in population

# 

